In [36]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import torch
from torch import nn, optim
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from torchinfo import summary
import pandas as pd
from torchvision import datasets
import os
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import confusion_matrix, accuracy_score
from torchvision import models, transforms, utils
import os
import torchvision
from zipfile import ZipFile
import opendatasets as od
from sklearn.decomposition import PCA
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## getting the dataset

In [15]:
if Path(".\\16P").is_dir():
    print("The dataset already exists. Skipping download....")
else:
    print("Downloading the dataset")
    od.download("https://www.kaggle.com/datasets/anshulmehtakaggl/60k-responses-of-16-personalities-test-mbt?select=16P.csv")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  raunakgola


Your Kaggle Key:

  ········


Dataset URL: https://www.kaggle.com/datasets/anshulmehtakaggl/60k-responses-of-16-personalities-test-mbt


100%|██████████████████████████████████████████████████████████████████████████████| 1.86M/1.86M [00:03<00:00, 488kB/s]

## first see the data how it looks like

In [17]:
df = pd.read_csv("C:/Users/Raunak Gola/image_outlier_2/60k-responses-of-16-personalities-test-mbt/16P1.csv")
df

,Response Id,You regularly make new friends.,You spend a lot of your free time exploring various random topics that pique your interest,Seeing other people cry can easily make you feel like you want to cry too,You often make a backup plan for a backup plan.,"You usually stay calm, even under a lot of pressure","At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know",You prefer to completely finish one project before starting another.,You are very sentimental.,You like to use organizing tools like schedules and lists.,...,You believe that pondering abstract philosophical questions is a waste of time.,"You feel more drawn to places with busy, bustling atmospheres than quiet, intimate places.",You know at first glance how someone is feeling.,You often feel overwhelmed.,You complete things methodically without skipping over any steps.,You are very intrigued by things labeled as controversial.,You would pass along a good opportunity if you thought someone else needed it more.,You struggle with deadlines.,You feel confident that things will work out for you.,Personality
0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,-1,0,0,0,0,0,ENFP
1,1,0,0,-2,-3,-1,2,-2,0,3,...,0,-2,0,2,0,-1,-1,-1,3,ISFP
2,2,0,0,2,0,-1,2,0,0,1,...,0,2,0,2,-1,0,1,2,1,INFJ
3,3,0,-1,3,-1,0,0,-2,0,-2,...,0,0,-1,-1,0,1,0,-2,-1,ISTP
4,4,0,0,-1,0,2,-1,-2,0,1,...,0,1,0,2,0,1,-1,2,-1,ENFJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59994,59994,0,-1,0,-3,-2,2,1,0,1,...,0,2,0,1,0,0,2,3,3,INFJ
59995,59995,0,0,-2,1,3,0,-3,0,2,...,0,1,0,0,0,0,2,0,-2,ESTP
59996,59996,0,0,1,0,-2,-2,0,0,-1,...,-1,2,0,-3,0,1,-1,0,-1,ISTP
59997,59997,0,0,1,-1,-2,-1,2,0,1,...,0,2,0,2,0,0,0,1,0,ISTJ


In [8]:
## preprocessing

In [18]:
df = df.drop(["Response Id"],axis=1)

In [24]:

# Assuming the last column is the label and the rest are features
features = df.iloc[:, :-1]
labels = df.iloc[:, -1]

# Apply PCA
n_components = 30  # Number of principal components to keep
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(features)

# Get the important columns (features)
important_columns = []
for i in range(n_components):
    loading_scores = pd.Series(pca.components_[i], index=features.columns)
    sorted_loading_scores = loading_scores.abs().sort_values(ascending=False)
    important_columns.append(sorted_loading_scores.index[0])

print(f"Important columns: {important_columns}")


Important columns: ['You enjoy watching people argue.', 'You often make a backup plan for a backup plan.', 'You usually stay calm, even under a lot of pressure', 'You often have a hard time understanding other people’s feelings.', 'You like to use organizing tools like schedules and lists.', 'Seeing other people cry can easily make you feel like you want to cry too', 'You would pass along a good opportunity if you thought someone else needed it more.', 'You often end up doing things at the last possible moment.', 'In your social circle, you are often the one who contacts your friends and initiates activities.', 'At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know', 'You are not too interested in discussing various interpretations and analyses of creative works.', 'You find it easy to empathize with a person whose experiences are very different from yours.', 'You often end up doing things at the last possible moment.', 'Your 

## getting the important columns

In [25]:
df_important = df[important_columns + [df.columns[-1]]]
df_important

,You enjoy watching people argue.,You often make a backup plan for a backup plan.,"You usually stay calm, even under a lot of pressure",You often have a hard time understanding other people’s feelings.,You like to use organizing tools like schedules and lists.,Seeing other people cry can easily make you feel like you want to cry too,You would pass along a good opportunity if you thought someone else needed it more.,You often end up doing things at the last possible moment.,"In your social circle, you are often the one who contacts your friends and initiates activities.","At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know",...,You are still bothered by mistakes that you made a long time ago.,"If your plans are interrupted, your top priority is to get back on track as soon as possible.",You would pass along a good opportunity if you thought someone else needed it more.,You often spend a lot of time trying to understand views that are very different from your own.,"If your plans are interrupted, your top priority is to get back on track as soon as possible.",You enjoy going to art museums.,You often have a hard time understanding other people’s feelings.,You often spend a lot of time trying to understand views that are very different from your own.,You are prone to worrying that things will take a turn for the worse.,Personality
0,0,0,0,-1,0,0,0,0,1,1,...,-1,0,0,0,0,0,-1,0,0,ENFP
1,1,-3,-1,2,3,-2,-1,-2,-1,2,...,2,1,-1,2,1,1,2,2,3,ISFP
2,2,0,-1,0,1,2,1,-1,-2,2,...,-3,0,1,-2,0,0,0,-2,0,INFJ
3,0,-1,0,3,-2,3,0,-2,0,0,...,0,0,0,0,0,0,3,0,1,ISTP
4,-1,0,2,-1,1,-1,-1,0,3,-1,...,-2,0,-1,-1,0,-2,-1,-1,2,ENFJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59994,2,-3,-2,1,1,0,2,-3,-1,2,...,0,1,2,-1,1,1,1,-1,2,INFJ
59995,0,1,3,0,2,-2,2,-1,-1,0,...,1,2,2,1,2,-3,0,1,-2,ESTP
59996,3,0,-2,0,-1,1,-1,0,-1,-2,...,1,2,-1,-2,2,1,0,-2,0,ISTP
59997,-2,-1,-2,2,1,1,0,1,-1,-1,...,0,-1,0,-1,-1,3,2,-1,-2,ISTJ


## encoding the label

In [27]:
# Convert text labels to numerical labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Get the mapping of labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f"Label mapping: {label_mapping}")

Label mapping: {'ENFJ': 0, 'ENFP': 1, 'ENTJ': 2, 'ENTP': 3, 'ESFJ': 4, 'ESFP': 5, 'ESTJ': 6, 'ESTP': 7, 'INFJ': 8, 'INFP': 9, 'INTJ': 10, 'INTP': 11, 'ISFJ': 12, 'ISFP': 13, 'ISTJ': 14, 'ISTP': 15}


In [29]:
df_important['Personality'] = labels_encoded

C:\Users\Raunak Gola\AppData\Local\Temp\ipykernel_27388\2635702290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_important['Personality'] = labels_encoded


In [33]:
df_important

,You enjoy watching people argue.,You often make a backup plan for a backup plan.,"You usually stay calm, even under a lot of pressure",You often have a hard time understanding other people’s feelings.,You like to use organizing tools like schedules and lists.,Seeing other people cry can easily make you feel like you want to cry too,You would pass along a good opportunity if you thought someone else needed it more.,You often end up doing things at the last possible moment.,"In your social circle, you are often the one who contacts your friends and initiates activities.","At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know",...,You are still bothered by mistakes that you made a long time ago.,"If your plans are interrupted, your top priority is to get back on track as soon as possible.",You would pass along a good opportunity if you thought someone else needed it more.,You often spend a lot of time trying to understand views that are very different from your own.,"If your plans are interrupted, your top priority is to get back on track as soon as possible.",You enjoy going to art museums.,You often have a hard time understanding other people’s feelings.,You often spend a lot of time trying to understand views that are very different from your own.,You are prone to worrying that things will take a turn for the worse.,Personality
0,0,0,0,-1,0,0,0,0,1,1,...,-1,0,0,0,0,0,-1,0,0,1
1,1,-3,-1,2,3,-2,-1,-2,-1,2,...,2,1,-1,2,1,1,2,2,3,13
2,2,0,-1,0,1,2,1,-1,-2,2,...,-3,0,1,-2,0,0,0,-2,0,8
3,0,-1,0,3,-2,3,0,-2,0,0,...,0,0,0,0,0,0,3,0,1,15
4,-1,0,2,-1,1,-1,-1,0,3,-1,...,-2,0,-1,-1,0,-2,-1,-1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59994,2,-3,-2,1,1,0,2,-3,-1,2,...,0,1,2,-1,1,1,1,-1,2,8
59995,0,1,3,0,2,-2,2,-1,-1,0,...,1,2,2,1,2,-3,0,1,-2,7
59996,3,0,-2,0,-1,1,-1,0,-1,-2,...,1,2,-1,-2,2,1,0,-2,0,15
59997,-2,-1,-2,2,1,1,0,1,-1,-1,...,0,-1,0,-1,-1,3,2,-1,-2,14


## train test split

In [39]:
X = df_important.iloc[:,:-1].values
y = df_important.iloc[:,-1].values

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## custom dataset class

In [44]:
class CustomDataset(Dataset):
    def __init__(self, features, labels, classes=None):
        self.features = features
        self.labels = labels
        self.classes = classes

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return torch.tensor(feature, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

## dataset and dataloader

In [49]:
classes = list(df["Personality"].unique())

In [50]:
# Create instances of the dataset
train_dataset = CustomDataset(X_train, y_train, classes=classes)
test_dataset = CustomDataset(X_test, y_test, classes=classes)
# Create a DataLoader to iterate through the dataset
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

print(f'Class names: {train_dataset.classes}')

Class names: ['ENFP', 'ISFP', 'INFJ', 'ISTP', 'ENFJ', 'INTJ', 'ENTJ', 'ESFP', 'INFP', 'INTP', 'ISTJ', 'ENTP', 'ISFJ', 'ESTJ', 'ESTP', 'ESFJ']


In [51]:
BATCH_SIZE = 64

In [52]:
print(f"Length of training dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of testing dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Length of training dataloader: 750 batches of 64
Length of testing dataloader: 188 batches of 64


In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
test_features, test_labels = next(iter(test_dataloader))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")

## setting up device agnostic code

In [ ]:
torch.manual_seed(5)
model = nn.Sequential(OrderedDict([
    ('Layer1',nn.Linear(30,64)),
    ('ReLU1',nn.ReLU()),
    ('Layer2',nn.Linear(64,64)),
    ('ReLU2',nn.ReLU()),
    ('Layer3',nn.Linear(64,64)),
    ('ReLU3',nn.ReLU()),
    ('Layer4',nn.Linear(64,16)),
]))

In [1]:
## model summary

In [ ]:
summary(model, input_size=(1,30,256,256))

In [2]:
## model state dict

In [3]:
#model.state_dict()

In [4]:
## Early Stopping Code

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, restore_best_weights=True, path='checkpoint.pt', verbose=False):
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.path = path
        self.early_stop = False

    def __call__(self, val_loss, model):
        loss = val_loss

        if self.best_loss is None:
            self.best_loss = loss
            torch.save(model.state_dict(), self.path)
            # print("saved model1")
        elif loss - self.best_loss >=  self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = loss
            if self.restore_best_weights:
                torch.save(model.state_dict(), self.path)
            
            self.counter = 0

In [5]:
## Setting up Loss function and Optimizer

In [ ]:
# Create loss function
criterion = nn.CrossEntropyLoss()

# Create optimizer
optimizer = torch.optim.Adam(params=model.parameters(), # optimize newly created model's parameters
                            lr=0.001)

In [ ]:
%%time
num_epochs = 15
trainloss = []
testloss = []
trainaccuracy = []
testaccuracy = []

## early stopping Instance
early_stopping = EarlyStopping(patience=10, verbose=True, delta = 0.001, path ="CNN_Classifier1.pt")
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_train_loss = 0.0
    correct_train = 0
    
    for train_images, train_labels in train_dataloader:
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(train_images.to(device))  # Forward pass
        train_loss = criterion(outputs, train_labels.to(device))  # Compute loss
        train_loss.backward()  # Backward pass
        optimizer.step()  # Optimize
        
        _, train_pred = torch.max(outputs.data,dim=-1) # predicted labels 
        correct_train += torch.eq(train_pred.to(device),train_labels.to(device)).sum()

        running_train_loss += train_loss.item() 
        train_accuracy = 100*(correct_train/len(train_dataloader.dataset))

    avg_train_loss = running_train_loss / len(train_dataloader)
    
    # Set model to evaluation mode
    correct_test = 0
    running_test_loss = 0.0
    model.eval()
    with torch.no_grad():
        for test_images, test_labels in test_dataloader:
            test_outputs = model(test_images.to(device))
            test_loss = criterion(test_outputs, test_labels.to(device))
            _, test_pred = torch.max(test_outputs.data, 1)
            running_test_loss += test_loss.item()
            correct_test += torch.eq(test_pred.to(device),test_labels.to(device)).sum()
            test_accuracy = 100*(correct_test/len(test_dataloader.dataset))

    avg_test_loss = running_test_loss / len(test_dataloader)

    trainloss.append(avg_train_loss)
    testloss.append(avg_test_loss)
    testaccuracy.append(test_accuracy)
    trainaccuracy.append(train_accuracy)
    print(f'Epoch [{epoch + 1}/{num_epochs}]| Train Loss: {avg_train_loss:.4f}| Train Accuracy: {train_accuracy:.2f}%| Test loss: {avg_test_loss:.4f}| Test Accuracy: {test_accuracy:.2f}%')
    early_stopping(avg_test_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

In [6]:
## Accuracy and Loss Curves

In [ ]:
e = []
for i in range(1,num_epochs):
    e.append(i)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(121)
plt.grid()
plt.plot(e,trainloss,color='#3980fa')
plt.plot(e,testloss,color='#fa9d39')
plt.title('Loss curves')
plt.legend(['train','test'])
plt.subplot(122)
plt.grid()
plt.plot(e,trainaccuracy,color='#3980fa')
plt.plot(e,testaccuracy,color='#fa9d39')
plt.title('Accuracy curves')
plt.legend(['train','test'])

In [7]:
## evaluate the model

In [ ]:
correct_test = 0
actual = []
predicted = []
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for test_images, test_labels in test_dataloader:
        test_outputs = model(test_images.to(device))
        test_loss = criterion(test_outputs, test_labels.to(device))
        _, test_pred = torch.max(test_outputs.data, 1)
        running_test_loss += test_loss.item()
        
        correct_test += torch.eq(test_pred.to(device),test_labels.to(device)).sum() # how many of them are correct 
        predicted += test_pred.tolist() #store predicted labels to list
        actual += test_labels.tolist() # store actual labels to list


In [ ]:
cm = confusion_matrix(actual,predicted)

In [ ]:
plt.figure(figsize=(10,9))
sns.heatmap(cm,cmap="plasma",annot=True,fmt='.5g')
print("The Acuuracy of the model is: ",round(accuracy_score(actual,predicted)*100,3)) 
# here 0 , 1, 2, represents cat, dog, human respectively